# Assignment 1: Dino Fun World

You, in your role as a burgeoning data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a couple questions about their park operations. In order to perform the requested analysis, they have provided you with a database containing information about one day of the park's operations.

### Provided Database

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkins', 'attractions', and 'sequences'. The information contained in each of these tables is listed below:

`checkins`:
    - Description: check-in data for all visitors for the day in the park. The data includes two types of check-ins, inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type
`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carussel, etc.
    - Fields: AttractionID, Name, Region, Category, type
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
The database is named 'dinofunworld.db' and is located in the 'readonly' directory of the Jupyter Notebook environment. It can be accessed at 'readonly/dinofunworld.db'.

### Questions to Answer

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.

Question 1: What is the most popular attraction to visit in the park?
Question 2: What ride (note that not all attractions are rides) has the longest visit time?
Question 3: Which Fast Food offering has the fewest visitors?
Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline.

#### Administrative Notes

This assignment will be graded by Coursera's grading system. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed.

In [10]:
import sqlite3
import math
from datetime import timedelta

In [11]:
db_filename = 'dinofunworld.db'
conn = sqlite3. connect(db_filename) 
c = conn.cursor()

In [12]:
# Graded Cell
# PartID:  NDnou
# Question 1: What is the most popular attraction to visit in the park?
# Notes: Your output should be the name of the attraction.
c.execute ("SELECT attraction, COUNT(attraction) AS 'Frequency' FROM checkin GROUP BY attraction ORDER BY COUNT(*) DESC LIMIT 1;")
attractionId = c.fetchall()[0][0]
c.execute("SELECT Name FROM attraction WHERE attractionid = "+str(attractionId)+";")
attractionName = c.fetchall()[0][0]
print(attractionName)

Atmosfear


In [13]:
# Graded Cell
# PartID: FXGHp
# Question 2: What ride (note that not all attractions are rides) has the longest average visit time?
# Notes: Your output should be the name of the ride.
c.execute("SELECT checkin.attraction, attraction.name, avg(checkin.duration) as 'Average Duration', attraction.category FROM checkin INNER JOIN attraction ON checkin.attraction = attraction.attractionid WHERE attraction.category LIKE '%rides%' GROUP by attraction ORDER BY 'Average Duration' DESC LIMIT 1;")
longestWait = c.fetchone()[1]
print(longestWait)

Flight of the Swingodon


In [14]:
# Graded Cell
# PartID: KALua
# Question 3: Which Fast Food offering in the park has the fewest visitors?
# Notes: Your output should be the name of the fast food offering.
c.execute("SELECT attraction.attractionid, COUNT(DISTINCT checkin.visitorid) AS 'uniqueVisitors', attraction.Name, attraction.type FROM checkin INNER JOIN attraction ON checkin.attraction = attraction.attractionid where attraction.type = 'Fast Food' GROUP BY attraction ORDER BY 'uniqueVisitors' ASC LIMIT 1;")
lowestFastFood = c.fetchall()[0][2]
print(lowestFastFood)

Theresaur Food Stop


In [15]:
# Graded Cell
# PartID: B0LUP
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and 
#  report the rides that appear in the Skyline. 
# Notes: Remember that in this case, higher visits is better and lower visit times are better. 
#  Your output should be formatted as an array listing the names of the rides in the Skyline.
skylineList = []
leastVisitTime = math.inf
attractionName2 = ""

c.execute ("SELECT attraction, COUNT(attraction) AS 'Frequency' FROM checkin GROUP BY attraction ORDER BY COUNT(*) DESC LIMIT 1;")
attractionId = c.fetchall()[0][0]
c.execute("SELECT Name FROM attraction WHERE attractionid = "+str(attractionId)+";")
attractionName = c.fetchall()[0][0]
skylineList.append(attractionName)

c.execute("SELECT checkin.attraction, attraction.name, avg(checkin.duration) as 'Average Visit Time', attraction.category FROM checkin INNER JOIN attraction ON checkin.attraction = attraction.attractionid WHERE attraction.category LIKE '%rides%' GROUP by attraction ORDER BY 'Average Visit Time' ASC LIMIT 1;")
attractionName1 = c.fetchall()[0][1]
skylineList.append(attractionName1)

c.execute("SELECT attractionid, name FROM attraction where category LIKE '%ride%'")
attRides = c.fetchall()

for attId, attName in attRides:
    c.execute("SELECT duration FROM checkin WHERE attraction = "+str(attId))
    attDur = c.fetchall()
    timeinSec = 0
    
    for i in range(len(attDur)):
        try:
            hrs, mins, sec = attDur[i][0].split(":")
            timeinSec = timeinSec + timedelta(int(hrs), int(mins), int(sec)).total_seconds()
        
        except:
            continue
        
    if (timeinSec < leastVisitTime):
        leastVisitTime = timeinSec
        attractionName2 = attName
        
skylineList.append(attractionName2)

print(skylineList)

['Atmosfear', 'Wrightiraptor Mountain', 'Dykesadactyl Thrill']
